In [1]:
# ライブラリーのインポート
import math
import torch
import torch.nn as nn

Positional Encoding
$$
PE_{(pos,2i)} = sin(pos/10000^{2i/d_{model}})
\\
PE_{(pos,2i+1)} = cos(pos/10000^{2i/d_{model}})
$$

In [7]:
#@title PositionalEncoding
class PositionalEncoding(nn.Module):
    # 初期化
    def __init__(self, d_model,context_size=512):
        super(PositionalEncoding,self).__init__()

        # ０行列の作成
        pe = torch.zeros(context_size,d_model)

        # 三角関数を使った計算ループ
        for pos in range(context_size):
          for i in range(0, d_model, 2):
            pe[pos, i] = math.sin(pos/(10000**(i/d_model)))
            pe[pos, i+1] = math.cos(pos/(10000**(i/d_model)))

        # 学習パラメーターの更新対象から外してクラス変数に確保(重要)
        self.register_buffer('pe', pe.unsqueeze(0))


    # 順伝播
    def forward(self,x):
      return self.pe[:,:x.size(1)]

In [5]:
# ハイパーパラメータ
batch_size = 1
d_model = 4
context_size = 5

In [8]:
# インスタンス化
pe = PositionalEncoding(d_model, context_size)

# 入力値のサンプリング
torch.random.manual_seed(0)
x = torch.randint(context_size,(batch_size,context_size))

# Positional encodingの実行
y = pe(x)

# 桁数をそろえて出力
print(x)
print(y)

tensor([[4, 4, 3, 0, 3]])
tensor([[[ 0.0000,  1.0000,  0.0000,  1.0000],
         [ 0.8415,  0.5403,  0.0100,  0.9999],
         [ 0.9093, -0.4161,  0.0200,  0.9998],
         [ 0.1411, -0.9900,  0.0300,  0.9996],
         [-0.7568, -0.6536,  0.0400,  0.9992]]])


In [9]:
# 埋め込みベクトルを
torch.manual_seed(0)
embedding = nn.Embedding(context_size, d_model)
embedding(x)

tensor([[[ 0.9318,  1.2590,  2.0050,  0.0537],
         [ 0.9318,  1.2590,  2.0050,  0.0537],
         [ 1.3894,  1.5863,  0.9463, -0.8437],
         [-1.1258, -1.1524, -0.2506, -0.4339],
         [ 1.3894,  1.5863,  0.9463, -0.8437]]], grad_fn=<EmbeddingBackward0>)

In [10]:
# 埋め込みベクトルへpositional encodingを追加して確認
embedding(x) + pe(x)

tensor([[[ 0.9318,  2.2590,  2.0050,  1.0537],
         [ 1.7733,  1.7993,  2.0150,  1.0537],
         [ 2.2987,  1.1702,  0.9663,  0.1561],
         [-0.9847, -2.1424, -0.2206,  0.5657],
         [ 0.6326,  0.9327,  0.9863,  0.1555]]], grad_fn=<AddBackward0>)